In [1]:
#Importing packages
import math as m
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics

pd.options.mode.chained_assignment = None  # default='warn'


pd.options.display.max_columns = 50
pd.options.display.max_rows = 99

In [2]:
# Games With out Chris Hansen in the line up
Boxdata = pd.read_excel('Tritons_GMS.xlsx', sheet_name='GMS_WO_Chris_12_5_to_12_22')
Boxdata = Boxdata.round({'AstTO': 2,'FG%':3,'FG%2':3,'FG%3':3,'FT%':3})

In [3]:
# Boxdata.loc[:,'FG%'] *= 100 
# Boxdata.loc[:,'FG%2'] *= 100
# Boxdata.loc[:,'FG%3'] *= 100 
# Boxdata.loc[:,'FT%'] *= 100 
Boxdata

,UCSDTritons,GP,Min,Pts,Ast,TO,AstTO,Stl,Blk,TtlReb,OREB,DREB,FGA,FGM,FGm,FG%,FGA2,FGM2,FGm2,FG%2,FGA3,FGM3,FGm3,FG%3,FTA,FTM,FTm,FT%
0,#1 Hugh Baxter,4,27,3,2,1,2.00,0,0,0,0,0,2,1,1,0.500,0,0,0,0.000,2,1,1,0.500,0,0,0,0.000
1,#2 Tyrell Roberts,4,141,69,18,7,2.57,1,1,11,1,10,53,22,31,0.415,19,9,10,0.474,34,13,21,0.382,12,12,0,1.000
2,#3 Gabe Hadley,4,85,44,5,3,1.67,3,0,7,0,7,29,14,15,0.483,3,2,1,0.667,26,12,14,0.462,4,4,0,1.000
3,#5 Marek Sullivan,4,123,53,10,5,2.00,11,2,29,6,23,28,20,8,0.714,20,16,4,0.800,8,4,4,0.500,14,9,5,0.643
4,#10 Connor Peterson,2,10,2,0,1,0.00,1,0,1,0,1,3,0,3,0.000,3,0,3,0.000,0,0,0,0.000,2,2,0,1.000
5,#11 Scott Everman,4,103,27,4,4,1.00,3,1,12,5,7,25,10,15,0.400,10,6,4,0.600,15,4,11,0.267,4,3,1,0.750
6,#14 Christian Oshita,4,137,71,3,7,0.43,5,0,23,6,17,52,25,27,0.481,23,15,8,0.652,29,10,19,0.345,13,11,2,0.846
7,#23 Kadeen Rasheed,3,15,2,0,1,0.00,1,0,2,0,2,2,1,1,0.500,2,1,1,0.500,0,0,0,0.000,0,0,0,0.000
8,#24 John Diener,1,5,0,0,0,0.00,0,0,0,0,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,0,0,0,0.000
9,#25 Martin Tombe,3,12,2,0,1,0.00,0,0,1,1,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,3,2,1,0.667


In [4]:
# I want to get a bunch of constants used in calculting some more advanced metrics
Tm_min = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','Min'].values[0])
Tm_FTA = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FTA'].values[0])
Tm_TOV = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','TO'].values[0])
Tm_FGA = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGA'].values[0])
Tm_PTS = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','Pts'].values[0])
Tm_AST = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','Ast'].values[0])
TM_FGM = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGM'].values[0])
Tm_FTM = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FTM'].values[0])
Tm_ORB = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','OREB'].values[0])
Tm_DRB = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','DREB'].values[0])
Tm_3PM = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGM3'].values[0])
Tm_3PA = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGA3'].values[0])
Tm_2PA = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGA2'].values[0])
Tm_2PM = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','FGM2'].values[0])
Tm_GP = float(Boxdata.loc[Boxdata.UCSDTritons == 'Team Totals','GP'].values[0])

In [5]:
Boxdata['TOV%'] = Boxdata.TO / (Boxdata.FGA + 0.44 * Boxdata.FTA + Boxdata.TO)
Boxdata = Boxdata.round({'TOV%': 3})
Boxdata['TRB%'] = ((Boxdata.FGA + 0.44 * Boxdata.FTA + Boxdata.TO) 
                   * (Tm_min / 5)) / (Boxdata.Min * (+ 0.44 * Tm_FTA + Tm_TOV))
Boxdata['eFG%'] = (Boxdata.FGM + (0.5 * Boxdata.FGM3)) / Boxdata.FGA
Boxdata = Boxdata.round({'eFG%': 3,'TRB%':3})
Boxdata['Usg%'] = ((Boxdata.FGA + 0.44*Boxdata.FTA+Boxdata.TO)*(Tm_min/5))/(Boxdata.Min*(Tm_FGA+0.44*Tm_FTA+Tm_TOV))
Boxdata = Boxdata.round({'Usg%': 3})
Boxdata['TS%'] =  Boxdata.Pts / (2 *  (Boxdata.FGA + 0.44 * Boxdata.FTA))
Boxdata = Boxdata.round({'TS%': 3})
Boxdata['TRB%'] = ((Boxdata.FGA + 0.44 * Boxdata.FTA + Boxdata.TO) 
                   * (Tm_min / 5)) / (Boxdata.Min * (+ 0.44 * Tm_FTA + Tm_TOV))
Boxdata = Boxdata.round({'eFG%': 3,'TRB%':3})
Boxdata['MPG'] = Boxdata['Min'].div(Boxdata.GP, axis=0)
Boxdata['FTAPG'] = Boxdata['FTA'].div(Boxdata.GP, axis=0)
Boxdata['FTMPG'] = Boxdata['FTM'].div(Boxdata.GP, axis=0)
Boxdata['AST%'] = Boxdata.Ast / (((Boxdata.Min / (Tm_min / 5)) * TM_FGM) - Boxdata.FGM)
Boxdata['APG'] =  Boxdata.Ast / Boxdata.GP
Boxdata = Boxdata.round({'APG': 3})
Boxdata['PPG'] =  Boxdata.Pts / Boxdata.GP
Boxdata['PP40'] = 40*Boxdata.Pts / Boxdata.Min
Boxdata['PPM'] = Boxdata.Pts / Boxdata.Min
Boxdata['FTA/FGA'] = Boxdata.FTA / Boxdata.FGA
Boxdata = Boxdata.round({'MPG': 1,'FTAPG':0,'FTMPG':0,'AST%':3,'PPG':3,'PPM':3,'PP40':3,'FTA/FGA':3})

In [6]:
# What amount of total FGA attempted by our team our 2P and 3P without Chris in lineup
two_pts_att = Tm_2PA/Tm_FGA
three_pts_att = Tm_3PA/Tm_FGA
print('% FGA that are 2P:',two_pts_att)
print('% FGA that are 3P:',three_pts_att)

% FGA that are 2P: 0.41935483870967744
% FGA that are 3P: 0.5806451612903226


In [7]:
Boxdata

,UCSDTritons,GP,Min,Pts,Ast,TO,AstTO,Stl,Blk,TtlReb,OREB,DREB,FGA,FGM,FGm,FG%,FGA2,FGM2,FGm2,FG%2,FGA3,FGM3,FGm3,FG%3,FTA,FTM,FTm,FT%,TOV%,TRB%,eFG%,Usg%,TS%,MPG,FTAPG,FTMPG,AST%,APG,PPG,PP40,PPM,FTA/FGA
0,#1 Hugh Baxter,4,27,3,2,1,2.00,0,0,0,0,0,2,1,1,0.500,0,0,0,0.000,2,1,1,0.500,0,0,0,0.000,0.333,0.055,0.750,0.013,0.750,6.8,0.0,0.0,0.023,0.50,0.750,4.444,0.111,0.000
1,#2 Tyrell Roberts,4,141,69,18,7,2.57,1,1,11,1,10,53,22,31,0.415,19,9,10,0.474,34,13,21,0.382,12,12,0,1.000,0.107,0.230,0.538,0.053,0.592,35.2,3.0,3.0,0.041,4.50,17.250,19.574,0.489,0.226
2,#3 Gabe Hadley,4,85,44,5,3,1.67,3,0,7,0,7,29,14,15,0.483,3,2,1,0.667,26,12,14,0.462,4,4,0,1.000,0.089,0.197,0.690,0.045,0.715,21.2,1.0,1.0,0.019,1.25,11.000,20.706,0.518,0.138
3,#5 Marek Sullivan,4,123,53,10,5,2.00,11,2,29,6,23,28,20,8,0.714,20,16,4,0.800,8,4,4,0.500,14,9,5,0.643,0.128,0.158,0.786,0.036,0.776,30.8,4.0,2.0,0.026,2.50,13.250,17.236,0.431,0.500
4,#10 Connor Peterson,2,10,2,0,1,0.00,1,0,1,0,1,3,0,3,0.000,3,0,3,0.000,0,0,0,0.000,2,2,0,1.000,0.205,0.242,0.000,0.055,0.258,5.0,1.0,1.0,0.000,0.00,1.000,8.000,0.200,0.667
5,#11 Scott Everman,4,103,27,4,4,1.00,3,1,12,5,7,25,10,15,0.400,10,6,4,0.600,15,4,11,0.267,4,3,1,0.750,0.130,0.148,0.480,0.034,0.504,25.8,1.0,1.0,0.012,1.00,6.750,10.485,0.262,0.160
6,#14 Christian Oshita,4,137,71,3,7,0.43,5,0,23,6,17,52,25,27,0.481,23,15,8,0.652,29,10,19,0.345,13,11,2,0.846,0.108,0.234,0.577,0.054,0.615,34.2,3.0,3.0,0.007,0.75,17.750,20.730,0.518,0.250
7,#23 Kadeen Rasheed,3,15,2,0,1,0.00,1,0,2,0,2,2,1,1,0.500,2,1,1,0.500,0,0,0,0.000,0,0,0,0.000,0.333,0.099,0.500,0.023,0.500,5.0,0.0,0.0,0.000,0.00,0.667,5.333,0.133,0.000
8,#24 John Diener,1,5,0,0,0,0.00,0,0,0,0,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,0,0,0,0.000,0.000,0.099,0.000,0.023,0.000,5.0,0.0,0.0,0.000,0.00,0.000,0.000,0.000,0.000
9,#25 Martin Tombe,3,12,2,0,1,0.00,0,0,1,1,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,3,2,1,0.667,0.301,0.137,0.000,0.031,0.431,4.0,1.0,1.0,0.000,0.00,0.667,6.667,0.167,3.000


In [8]:
Boxdata_Nototals = Boxdata.drop([11])
Boxdata_Nototals

,UCSDTritons,GP,Min,Pts,Ast,TO,AstTO,Stl,Blk,TtlReb,OREB,DREB,FGA,FGM,FGm,FG%,FGA2,FGM2,FGm2,FG%2,FGA3,FGM3,FGm3,FG%3,FTA,FTM,FTm,FT%,TOV%,TRB%,eFG%,Usg%,TS%,MPG,FTAPG,FTMPG,AST%,APG,PPG,PP40,PPM,FTA/FGA
0,#1 Hugh Baxter,4,27,3,2,1,2.00,0,0,0,0,0,2,1,1,0.500,0,0,0,0.000,2,1,1,0.500,0,0,0,0.000,0.333,0.055,0.750,0.013,0.750,6.8,0.0,0.0,0.023,0.50,0.750,4.444,0.111,0.000
1,#2 Tyrell Roberts,4,141,69,18,7,2.57,1,1,11,1,10,53,22,31,0.415,19,9,10,0.474,34,13,21,0.382,12,12,0,1.000,0.107,0.230,0.538,0.053,0.592,35.2,3.0,3.0,0.041,4.50,17.250,19.574,0.489,0.226
2,#3 Gabe Hadley,4,85,44,5,3,1.67,3,0,7,0,7,29,14,15,0.483,3,2,1,0.667,26,12,14,0.462,4,4,0,1.000,0.089,0.197,0.690,0.045,0.715,21.2,1.0,1.0,0.019,1.25,11.000,20.706,0.518,0.138
3,#5 Marek Sullivan,4,123,53,10,5,2.00,11,2,29,6,23,28,20,8,0.714,20,16,4,0.800,8,4,4,0.500,14,9,5,0.643,0.128,0.158,0.786,0.036,0.776,30.8,4.0,2.0,0.026,2.50,13.250,17.236,0.431,0.500
4,#10 Connor Peterson,2,10,2,0,1,0.00,1,0,1,0,1,3,0,3,0.000,3,0,3,0.000,0,0,0,0.000,2,2,0,1.000,0.205,0.242,0.000,0.055,0.258,5.0,1.0,1.0,0.000,0.00,1.000,8.000,0.200,0.667
5,#11 Scott Everman,4,103,27,4,4,1.00,3,1,12,5,7,25,10,15,0.400,10,6,4,0.600,15,4,11,0.267,4,3,1,0.750,0.130,0.148,0.480,0.034,0.504,25.8,1.0,1.0,0.012,1.00,6.750,10.485,0.262,0.160
6,#14 Christian Oshita,4,137,71,3,7,0.43,5,0,23,6,17,52,25,27,0.481,23,15,8,0.652,29,10,19,0.345,13,11,2,0.846,0.108,0.234,0.577,0.054,0.615,34.2,3.0,3.0,0.007,0.75,17.750,20.730,0.518,0.250
7,#23 Kadeen Rasheed,3,15,2,0,1,0.00,1,0,2,0,2,2,1,1,0.500,2,1,1,0.500,0,0,0,0.000,0,0,0,0.000,0.333,0.099,0.500,0.023,0.500,5.0,0.0,0.0,0.000,0.00,0.667,5.333,0.133,0.000
8,#24 John Diener,1,5,0,0,0,0.00,0,0,0,0,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,0,0,0,0.000,0.000,0.099,0.000,0.023,0.000,5.0,0.0,0.0,0.000,0.00,0.000,0.000,0.000,0.000
9,#25 Martin Tombe,3,12,2,0,1,0.00,0,0,1,1,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,3,2,1,0.667,0.301,0.137,0.000,0.031,0.431,4.0,1.0,1.0,0.000,0.00,0.667,6.667,0.167,3.000


In [9]:
Boxdata.sort_values(by=['TS%'], ascending=False)

,UCSDTritons,GP,Min,Pts,Ast,TO,AstTO,Stl,Blk,TtlReb,OREB,DREB,FGA,FGM,FGm,FG%,FGA2,FGM2,FGm2,FG%2,FGA3,FGM3,FGm3,FG%3,FTA,FTM,FTm,FT%,TOV%,TRB%,eFG%,Usg%,TS%,MPG,FTAPG,FTMPG,AST%,APG,PPG,PP40,PPM,FTA/FGA
3,#5 Marek Sullivan,4,123,53,10,5,2.00,11,2,29,6,23,28,20,8,0.714,20,16,4,0.800,8,4,4,0.500,14,9,5,0.643,0.128,0.158,0.786,0.036,0.776,30.8,4.0,2.0,0.026,2.50,13.250,17.236,0.431,0.500
0,#1 Hugh Baxter,4,27,3,2,1,2.00,0,0,0,0,0,2,1,1,0.500,0,0,0,0.000,2,1,1,0.500,0,0,0,0.000,0.333,0.055,0.750,0.013,0.750,6.8,0.0,0.0,0.023,0.50,0.750,4.444,0.111,0.000
10,#32 Mickey Howell,4,142,34,29,9,3.22,9,1,14,5,9,21,12,9,0.571,11,6,5,0.545,10,6,4,0.600,6,4,2,0.667,0.276,0.114,0.714,0.026,0.719,35.5,2.0,1.0,0.064,7.25,8.500,9.577,0.239,0.286
2,#3 Gabe Hadley,4,85,44,5,3,1.67,3,0,7,0,7,29,14,15,0.483,3,2,1,0.667,26,12,14,0.462,4,4,0,1.000,0.089,0.197,0.690,0.045,0.715,21.2,1.0,1.0,0.019,1.25,11.000,20.706,0.518,0.138
11,Team Totals,4,160,307,71,39,1.82,34,5,100,24,76,217,105,112,0.484,91,55,36,0.604,126,50,76,0.397,58,47,11,0.810,0.139,0.873,0.599,0.200,0.633,40.0,14.0,12.0,0.169,17.75,76.750,76.750,1.919,0.267
6,#14 Christian Oshita,4,137,71,3,7,0.43,5,0,23,6,17,52,25,27,0.481,23,15,8,0.652,29,10,19,0.345,13,11,2,0.846,0.108,0.234,0.577,0.054,0.615,34.2,3.0,3.0,0.007,0.75,17.750,20.730,0.518,0.250
1,#2 Tyrell Roberts,4,141,69,18,7,2.57,1,1,11,1,10,53,22,31,0.415,19,9,10,0.474,34,13,21,0.382,12,12,0,1.000,0.107,0.230,0.538,0.053,0.592,35.2,3.0,3.0,0.041,4.50,17.250,19.574,0.489,0.226
5,#11 Scott Everman,4,103,27,4,4,1.00,3,1,12,5,7,25,10,15,0.400,10,6,4,0.600,15,4,11,0.267,4,3,1,0.750,0.130,0.148,0.480,0.034,0.504,25.8,1.0,1.0,0.012,1.00,6.750,10.485,0.262,0.160
7,#23 Kadeen Rasheed,3,15,2,0,1,0.00,1,0,2,0,2,2,1,1,0.500,2,1,1,0.500,0,0,0,0.000,0,0,0,0.000,0.333,0.099,0.500,0.023,0.500,5.0,0.0,0.0,0.000,0.00,0.667,5.333,0.133,0.000
9,#25 Martin Tombe,3,12,2,0,1,0.00,0,0,1,1,0,1,0,1,0.000,0,0,0,0.000,1,0,1,0.000,3,2,1,0.667,0.301,0.137,0.000,0.031,0.431,4.0,1.0,1.0,0.000,0.00,0.667,6.667,0.167,3.000


In [10]:
# This resulting data frame is comprised of players whom have played in more that 9 games
# rslt_df = Boxdata.loc[Boxdata['GP'] > 9].loc[Boxdata['Min']>100]
# rslt_df

In [11]:
Boxdata2 = pd.read_excel('Tritons_GMS.xlsx', sheet_name='First8gms_withChris')
Boxdata2 = Boxdata2.round({'AstTO': 2,'FG%':3,'FG%2':3,'FG%3':3,'FT%':3})
Tm_min = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','Min'].values[0])
Tm_FTA = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FTA'].values[0])
Tm_TOV = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','TO'].values[0])
Tm_FGA = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGA'].values[0])
Tm_PTS = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','Pts'].values[0])
Tm_AST = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','Ast'].values[0])
TM_FGM = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGM'].values[0])
Tm_FTM = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FTM'].values[0])
Tm_ORB = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','OREB'].values[0])
Tm_DRB = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','DREB'].values[0])
Tm_3PM = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGM3'].values[0])
Tm_3PA = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGA3'].values[0])
Tm_2PA = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGA2'].values[0])
Tm_2PM = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','FGM2'].values[0])
Tm_GP = float(Boxdata2.loc[Boxdata2.UCSDTritons == 'Team Totals','GP'].values[0])

In [13]:
Boxdata2['TOV%'] = Boxdata2.TO / (Boxdata2.FGA + 0.44 * Boxdata2.FTA + Boxdata2.TO)
Boxdata2 = Boxdata2.round({'TOV%': 3})
Boxdata2['TRB%'] = ((Boxdata2.FGA + 0.44 * Boxdata2.FTA + Boxdata2.TO) 
                   * (Tm_min / 5)) / (Boxdata2.Min * (+ 0.44 * Tm_FTA + Tm_TOV))
Boxdata2['eFG%'] = (Boxdata2.FGM + (0.5 * Boxdata2.FGM3)) / Boxdata2.FGA
Boxdata2 = Boxdata2.round({'eFG%': 3,'TRB%':3})
Boxdata2['Usg%'] = ((Boxdata2.FGA + 0.44*Boxdata2.FTA+Boxdata2.TO)*(Tm_min/5))/(Boxdata2.Min*(Tm_FGA+0.44*Tm_FTA+Tm_TOV))
Boxdata2 = Boxdata2.round({'Usg%': 3})
Boxdata2['TS%'] =  Boxdata2.Pts / (2 *  (Boxdata2.FGA) + (0.44 * Boxdata2.FTA))
Boxdata2 = Boxdata2.round({'TS%': 3})
Boxdata2['TRB%'] = ((Boxdata2.FGA + 0.44 * Boxdata2.FTA + Boxdata2.TO) 
                   * (Tm_min / 5)) / (Boxdata2.Min * (+ 0.44 * Tm_FTA + Tm_TOV))
Boxdata2 = Boxdata2.round({'eFG%': 3,'TRB%':3})
Boxdata2['MPG'] = Boxdata2['Min'].div(Boxdata2.GP, axis=0)
Boxdata2['FTAPG'] = Boxdata2['FTA'].div(Boxdata2.GP, axis=0)
Boxdata2['FTMPG'] = Boxdata2['FTM'].div(Boxdata2.GP, axis=0)
Boxdata2['AST%'] = Boxdata2.Ast / (((Boxdata2.Min / (Tm_min / 5)) * TM_FGM) - Boxdata2.FGM)
Boxdata2['APG'] =  Boxdata2.Ast / Boxdata2.GP
Boxdata2 = Boxdata2.round({'APG': 3})
Boxdata2['PPG'] =  Boxdata2.Pts / Boxdata2.GP
Boxdata2['PP40'] = 40*Boxdata2.Pts / Boxdata2.Min
Boxdata2['PPM'] = Boxdata2.Pts / Boxdata2.Min
Boxdata2['FTA/FGA'] = Boxdata2.FTA / Boxdata2.FGA
Boxdata2 = Boxdata2.round({'MPG': 1,'FTAPG':0,'FTMPG':0,'AST%':3,'PPG':3,'PPM':3,'PP40':3,'FTA/FGA':3})
Boxdata2_Nototals = Boxdata2.drop([15])
Boxdata2

,UCSDTritons,GP,Min,Pts,Ast,TO,AstTO,Stl,Blk,TtlReb,OREB,DREB,FGA,FGM,FGm,FG%,FGA2,FGM2,FGm2,FG%2,FGA3,FGM3,FGm3,FG%3,FTA,FTM,FTm,FTP,TOV%,TRB%,eFG%,Usg%,TS%,MPG,FTAPG,FTMPG,AST%,APG,PPG,PP40,PPM,FTA/FGA
0,#0 Lunden Taylor,3,13,0,1,3,0.00,0,0,2,0,2,3,0,3,0.000,1,0,1,0.000,2,0,2,0.000,0,0,0,0.000000,0.500,0.250,0.000,0.049,0.000,4.3,0.0,0.0,0.019,0.333,0.000,0.000,0.000,0.000
1,#1 Hugh Baxter,7,69,20,6,3,0.50,2,0,11,3,8,30,12,18,0.410,17,7,10,0.591,13,5,8,0.250,9,8,1,0.875000,0.081,0.290,0.483,0.056,0.313,9.9,1.0,1.0,0.022,0.857,2.857,11.594,0.290,0.300
2,#2 Tyrell Roberts,8,200,121,25,12,12.50,7,0,23,1,22,92,42,50,0.469,35,22,13,0.579,53,18,35,0.364,5,4,1,0.750000,0.113,0.287,0.554,0.056,0.650,25.0,1.0,0.0,0.033,3.125,15.125,24.200,0.605,0.054
3,#3 Gabe Hadley,8,87,41,5,8,3.00,2,1,17,6,11,34,18,16,0.529,15,12,3,0.875,16,5,11,0.339,2,2,0,1.000000,0.187,0.267,0.603,0.052,0.595,10.9,0.0,0.0,0.015,0.625,5.125,18.851,0.471,0.059
4,#5 Marek Sullivan,8,119,49,16,2,4.00,8,3,25,10,15,31,22,9,0.691,22,19,3,0.861,9,3,6,0.375,5,5,0,1.000000,0.057,0.160,0.758,0.031,0.763,14.9,1.0,1.0,0.035,2.000,6.125,16.471,0.412,0.161
5,#11 Scott Everman,8,142,57,11,6,5.50,2,2,14,3,11,43,21,22,0.562,16,13,3,0.820,22,8,14,0.320,4,2,2,0.500000,0.118,0.193,0.581,0.038,0.649,17.8,0.0,0.0,0.020,1.375,7.125,16.056,0.401,0.093
6,#12 Justin Pratt,5,24,11,0,0,0.00,1,0,2,0,2,15,10,5,0.775,8,5,3,0.750,7,5,2,0.834,3,1,2,0.250000,0.000,0.368,0.833,0.072,0.351,4.8,1.0,0.0,0.000,0.000,2.200,18.333,0.458,0.200
7,#14 Christian Oshita,8,208,134,19,11,11.34,5,1,50,15,35,78,44,34,0.526,32,21,11,0.665,42,20,22,0.440,23,19,4,0.752800,0.111,0.258,0.692,0.050,0.807,26.0,3.0,2.0,0.024,2.375,16.750,25.769,0.644,0.295
8,#15 Jeremy Johnson,3,12,0,2,0,0.00,0,0,2,2,0,1,0,1,0.000,1,0,1,0.000,0,0,0,0.000,0,0,0,0.000000,0.000,0.045,0.000,0.009,0.000,4.0,0.0,0.0,0.041,0.667,0.000,0.000,0.000,0.000
9,#23 Kadeen Rasheed,7,67,16,7,2,0.00,3,0,9,3,6,12,5,7,0.533,3,3,0,1.000,9,2,7,0.300,5,3,2,0.583500,0.123,0.131,0.500,0.025,0.611,9.6,1.0,0.0,0.026,1.000,2.286,9.552,0.239,0.417


In [ ]:
# What amount of total FGA attempted by our team our 2P and 3P with Chris in lineup
two_pts_att = Tm_2PA/Tm_FGA
three_pts_att = Tm_3PA/Tm_FGA
print('% FGA that are 2P:',two_pts_att)
print('% FGA that are 3P:',three_pts_att)
two_pts_att+three_pts_att

In [ ]:
rslt_df = Boxdata[Boxdata["UCSDTritons"].isin(["Team Totals"])]

In [ ]:
rslt_df = rslt_df.append(Boxdata2[Boxdata2["UCSDTritons"].isin(["Team Totals"])],sort=False)

In [ ]:
rslt_df['PP40min'] = 40*rslt_df.Pts / rslt_df.Min
rslt_df

In [ ]:
rslt_df['TS%'] =  rslt_df.Pts / (2 *  (rslt_df.FGA + 0.44 * rslt_df.FTA))
rslt_df = rslt_df.round({'TS%': 3})

In [ ]:
plt.scatter(x=Boxdata['Min'], y=Boxdata['TS%'])


In [ ]:
Boxdata2

In [ ]:
# We need some colors to make sure all players have a unique color

import csv

with open('colors.csv', 'r') as f:
    reader = csv.reader(f)
    your_list = list(reader)

print(your_list)

In [ ]:
# code used to flatten a nested list
def flatten(lst):
    flat = []
    [flat.extend(flatten(i)) if isinstance(i,list) else flat.append(i) for i in lst]
    return flat

In [ ]:
colors_lst = flatten(your_list)

In [ ]:
colors_lst[0]='Brown'

In [ ]:
colors_lst[1]='Khaki'
colors_lst

In [ ]:
fig=plt.figure()
for _, row in Boxdata_Nototals.iterrows():
    plt.plot(row["MPG"], row["TS%"], 'o', ms=row.GP,
             label=row.UCSDTritons, c=colors_lst[_])
plt.xlabel("MPG"),plt.ylabel('TS%'),plt.title("TS% Performance from 12/5/19 to 12/22/19")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.figure(figsize=(20,10))
fig.savefig('MBB_TS%_12_5', bbox_extra_artists=(lgd,), bbox_inches='tight',dpi=180)

In [ ]:
# df1 = Boxdata_Nototals.drop(columns = ['GP','Min','Pts','Ast','TO','AstTO','Stl','Blk','TtlReb','OREB','DREB','FGA','FGM','FGm','FG%','FGA2','FGM2','FGm2','FG%2','FGA3','FGM3','FGm3','FG%3','FTA','FTM','FTm','FT%','TOV%','TRB%','eFG%','Usg%'])

In [ ]:
fig=plt.figure()
for _, row in Boxdata2_Nototals.iterrows():
    plt.plot(row["MPG"], row["TS%"], 'o', ms=row.GP,
             label=row.UCSDTritons, c=colors_lst[_])
plt.xlabel("MPG"),plt.ylabel('TS%'),plt.title("TS% Performance from 11/4/19 to 11/30/19")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.figure(figsize=(40,20))
fig.savefig('MBB_TS%_11_4', bbox_extra_artists=(lgd,), bbox_inches='tight',dpi=180)

In [ ]:
pd.concat(
    [Boxdata['FTAPG'], Boxdata['PPG'], 
     Boxdata['APG']],
    axis=1).plot.bar()


In [ ]:
# plt.figure(figsize=(10, 6))
# # sns.catplot(x='class', y='survival rate', hue='sex', data=df, kind='bar')

# chart = sns.catplot(x="PPG", hue='UCSDTritons', y=100, data=Boxdata_Nototals)
# chart.set_xticklabels(chart.get_xticklabels(),rotation=45)
# plt.show()

In [ ]:
sns.factorplot("PPG", "MPG","AstTO", col="UCSDTritons", data=Boxdata_Nototals, kind="bar")


In [ ]:
Boxdata_Nototals.plot(x="UCSDTritons", y=["TS%","AST%","Usg%"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_ast_to_usg_wo44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata_Nototals.plot(x="UCSDTritons", y=["AstTO","PPG","FTAPG"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_ASTto_PPG_wo44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata_Nototals.plot(x="UCSDTritons", y=["MPG","AstTO","PPG","FTAPG"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_MPG_PPG_wo44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata2_Nototals.plot(x="UCSDTritons", y=["TS%","AST%","Usg%"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_ast_to_usg_w44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata2_Nototals.plot(x="UCSDTritons", y=["AstTO","PPG","FTAPG"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_ASTto_PPG_w44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata2_Nototals.plot(x="UCSDTritons", y=["AstTO","PPG","FTAPG"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
# plt.savefig('MBB_ASTTO_PPG_w44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata_Nototals.plot(x="UCSDTritons", y=["TRB%","TS%","Usg%"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_trb_ts_usg_wo44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)

In [ ]:
Boxdata_Nototals.plot(x="UCSDTritons", y=["Blk","TS%","Usg%"], kind="bar")
lgd = plt.legend(loc='upper left', bbox_to_anchor=(1,1))
plt.savefig('MBB_stl_blk_dreb_wo44.png',bbox_extra_artists=(lgd,),bbox_inches='tight',dpi=130)